## 21. Image Pyramids

일반적으로는 고정된 이미지 사이즈를 작업을 하지만, 때때로 동일한 이미지에 대해서 다양한 사이즈를 가지고 작업을 해야 하는 경우가 있습니다. 만일, 이미지에서 얼굴을 찾을 경우에 얼굴의 사이즈를 확신할 수 없습니다. 이럴 경우에는 원본 이미지에 대한 다양한 사이즈에서 얼굴을 찾는다면 좀더 정확하고 확실한 이미지를 찾을 수 있습니다. 이 처럼 동일 이미지의 서로 다른 사이즈의 set을 Image Pyramids라고 합니다(가장 아래에 가장 큰 해상도를 놓고 점점 줄여가면서 쌓아가는 형태입니다.)

Pyramid, or pyramid representation, is a type of multi-scale signal representation developed by the computer vision, image processing and signal processing communities, in which a signal or an image is subject to repeated smoothing and subsampling.

<img style="float: left;" src="pic/fig_05.png" width="350">

opencv에는 두 종류의 Pyramid
1. Gaussian Pyramid
2. Laplacian Pyramid

Gaussian Pyramid 의 pyrDown 필터 (M x N 사이지의 이미지는 M/2 X N/2 가 적용되어 크기가  1/4사이즈로 줄어들게 됩니다.)

In [ ]:
import cv2
import numpy as np
img = cv2.imread("pic/lena.jpg")
lr = cv2.pyrDown(img)

cv2.imshow("Original image", img)
cv2.imshow("pyrDown 1 image", lr)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
img = cv2.imread("pic/lena.jpg")
lr1 = cv2.pyrDown(img)
lr2 = cv2.pyrDown(lr1)

cv2.imshow("Original image", img)
cv2.imshow("pyrDown 1 image", lr1)
cv2.imshow("pyrDown 2 image", lr2)

cv2.waitKey(0)
cv2.destroyAllWindows()

pyrUP 을 사용해보자

In [ ]:
import cv2
import numpy as np
img = cv2.imread("pic/lena.jpg")
lr1 = cv2.pyrDown(img)
lr2 = cv2.pyrDown(lr1)
hr1 = cv2.pyrUp(lr2)

cv2.imshow("Original image", img)
cv2.imshow("pyrDown 1 image", lr1)
cv2.imshow("pyrDown 2 image", lr2)
cv2.imshow("pyrUp 1 image", hr1)

cv2.waitKey(0)
cv2.destroyAllWindows()

lr1 과 hr1을 비교해보면 동일하지 않다.  
lr2로 pyrDown될때 lr2의 Information 일부를 잃어버렸다.  
따라서, pyrUp 해도 잃어버린 Information을 복구할 수는 없다.

반복문을 사용하여 여러레벨의 이미지를 만들자.

In [ ]:
import cv2
import numpy as np
img = cv2.imread("pic/lena.jpg")
layer = img.copy()
gaussian_pyramid_list = [layer]

for i in range(3):
    layer = cv2.pyrDown(layer)
    gaussian_pyramid_list.append(layer)
    cv2.imshow(str(i), layer)

cv2.imshow("Original image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Laplacian Pyramid는 Gaussian Pyramid에서 만들어 집니다.


Laplacian Pyramid image는   
1) 어떤 레벨의 image와   
2) 한 단계 낮은 resolution의 image에 pyrUp을 적용한 image  
와의 차이를 나타낸다.

이렇게 하면 외곽선이 남게 된다.

In [ ]:
import cv2
import numpy as np
img = cv2.imread("pic/lena.jpg")
layer = img.copy()
gaussian_pyramid_list = [layer]

for i in range(6):
    layer = cv2.pyrDown(layer)
    gaussian_pyramid_list.append(layer)
    #cv2.imshow(str(i), layer)

layer = gaussian_pyramid_list[5]
cv2.imshow('upper level Gaussian Pyramid', layer)
laplacian_pyramid_list = [layer]

for i in range(5, 0, -1):
    gaussian_extended = cv2.pyrUp(gaussian_pyramid_list[i])
    laplacian = cv2.subtract(gaussian_pyramid_list[i-1], gaussian_extended)
    cv2.imshow(str(i), laplacian)

cv2.imshow("Original image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 22. Image Blending using Pyramids

In [ ]:
import cv2
import numpy as np
apple = cv2.imread('pic/apple.jpg')
orange = cv2.imread('pic/orange.jpg')
print(apple.shape)
print(orange.shape)

cv2.imshow("apple", apple)
cv2.imshow("orange", orange)

cv2.waitKey(0)
cv2.destroyAllWindows()

두 그림의 사이즈가 같다.
사과 왼쪽 반절과 오렌지 오른쪽 반절을 합성하자.  
그냥 오려 붙이기를 해보자.

In [ ]:
import cv2
import numpy as np
apple = cv2.imread('pic/apple.jpg')
orange = cv2.imread('pic/orange.jpg')
#print(apple.shape)
#print(orange.shape)
apple_orange = np.hstack((apple[:, :256], orange[:, 256:]))

cv2.imshow("apple", apple)
cv2.imshow("orange", orange)
cv2.imshow("apple_orange", apple_orange)

cv2.waitKey(0)
cv2.destroyAllWindows()

image blending 절차
1. Load thr two images of apple and orang.
2. Find the Gaussian Pyramids for apple and orange (in this example, number of level is 6)
3. From Gaussian Pyramids, find their Laplacian Pyramids
4. Now join the left half of apple and right half of orange in each level of Laplacian Pyramids.
5. Fianlly from this joint image pyramids, reconstruct the originalimage.

In [ ]:
import cv2
import numpy as np
apple = cv2.imread('pic/apple.jpg')
orange = cv2.imread('pic/orange.jpg')
#print(apple.shape)
#print(orange.shape)
apple_orange = np.hstack((apple[:, :256], orange[:, 256:]))

# generate Gaussian pyramid for apple
apple_copy = apple.copy()
gp_apple = [apple_copy]
for i in range(6):
    apple_copy = cv2.pyrDown(apple_copy)
    gp_apple.append(apple_copy)

# generate Gaussian pyramid for orange
orange_copy = orange.copy()
gp_orange = [orange_copy]
for i in range(6):
    orange_copy = cv2.pyrDown(orange_copy)
    gp_orange.append(orange_copy)
    
# generate Laplacian Pyramid for apple
apple_copy = gp_apple[5]
lp_apple = [apple_copy]
for i in range(5, 0, -1):
    gaussian_expanded = cv2.pyrUp(gp_apple[i])
    laplacian = cv2.subtract(gp_apple[i-1], gaussian_expanded)
    lp_apple.append(laplacian)
    
# generate Laplacian Pyramid for orange
orange_copy = gp_orange[5]
lp_orange = [orange_copy]
for i in range(5, 0, -1):
    gaussian_expanded = cv2.pyrUp(gp_orange[i])
    laplacian = cv2.subtract(gp_orange[i-1], gaussian_expanded)
    lp_orange.append(laplacian)

# Now add left and right halves of images in each level
apple_orange_pyramid = []
n = 0
for apple_lap, orange_lap in zip(lp_apple, lp_orange):
    n += 1
    cols, rows, ch = apple_lap.shape
    laplacian = np.hstack((apple_lap[:, 0:int(cols/2)], orange_lap[:, int(cols/2):]))
    apple_orange_pyramid.append(laplacian)

# now reconstruct
apple_orange_reconstruct = apple_orange_pyramid[0]
for i in range(1, 6):
    apple_orange_reconstruct = cv2.pyrUp(apple_orange_reconstruct)
    apple_orange_reconstruct = cv2.add(apple_orange_pyramid[i], apple_orange_reconstruct)

cv2.imshow("apple", apple)
cv2.imshow("orange", orange)
cv2.imshow("apple_orange", apple_orange)
cv2.imshow("apple_orange_reconstruct", apple_orange_reconstruct)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 23. Find and Draw Contours

Contours란 동일한 색 또는 동일한 강도를 가지고 있는 영역의 경계선을 연결한 선입니다.   
우리가 자주 보는 것으로는 등고선이나 일기예보에서 볼 수 있습니다.

대상의 외형을 파악하는데 유용하게 사용이 됩니다.

- 정확도를 높히기 위해서 Binary Image를 사용합니다. 
- threshold나 canny edge를 선처리로 수행합니다.
- cv2.findContours() 함수는 원본 이미지를 직접 수정하기 때문에, 원본 이미지를 보존 하려면 Copy해서 사용해야 합니다.
- OpenCV에서는 contours를 찾는 것은 검은색 배경에서 하얀색 대상을 찾는 것과 비슷합니다. 그래서 대상은 흰색, 배경은 검은색으로 해야 합니다.

#### OpenCV에서 contours를 찾을 때

```
cv2.findContours(image, mode, method[, contours[, hierarchy[, offset]]]) → image, contours, hierarchy
```


```
Parameters:

image – 8-bit single-channel image. binary image.

mode – contours를 찾는 방법
    cv2.RETR_EXTERNAL : contours line중 가장 바같쪽 Line만 찾음.
    cv2.RETR_LIST : 모든 contours line을 찾지만, hierachy 관계를 구성하지 않음.
    cv2.RETR_CCOMP : 모든 contours line을 찾으며, hieracy관계는 2-level로 구성함.
    cv2.RETR_TREE : 모든 contours line을 찾으며, 모든 hieracy관계를 구성함.

method – contours를 찾을 때 사용하는 근사치 방법
    cv2.CHAIN_APPROX_NONE : 모든 contours point를 저장.
    cv2.CHAIN_APPROX_SIMPLE : contours line을 그릴 수 있는 point 만 저장. (ex; 사각형이면 4개 point)
    cv2.CHAIN_APPROX_TC89_L1 : contours point를 찾는 algorithm
    cv2.CHAIN_APPROX_TC89_KCOS : contours point를 찾는 algorithm

Returns: image, contours , hierachy
```

Method에 대해서 설명을 하면 사각형의 contours line을 그릴 때, cv2.CHAIN_APPROX_NONE 는 모든 point를 저장하고 cv2.CHAIN_APPROX_SIMPLE 는 4개의 point만을 저장하여 메모리를 절약합니다.

```
contours[0].shape #cv2.CHAIN_APPROX_SIMPLE(4 point)
(4, 1, 2)
```

```
contours[0].shape #CHAIN_APPROX_NONE(1308 point)
(1308, 1, 2)
```

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/shapes5.jpg')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow('Image', img)
cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/shapes5.jpg')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(imgray, 50, 255, 0)
#contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

imgGrey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thrash = cv2.threshold(imgGrey, 240, 255, cv2.THRESH_BINARY) #240
contours, _ = cv2.findContours(thrash, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

# Contours is a python list of all the contours in the image
# Each individual contour is a numpy array of (x,y) coordinates of boundary points of the objects.

print("Number of contours = " + str(len(contours)))
print(contours[0].shape)
#print(contours)
#cv2.imshow('Image', img)
#cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/shapes5.jpg')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 240, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Contours is a python list of all the contours in the image
# Each individual contour is a numpy array of (x,y) coordinates of boundary points of the objects.

print("Number of contours = " + str(len(contours)))
print(contours[0].shape)
print(contours)
#cv2.imshow('Image', img)
#cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/opencv-logo.png')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow('Image', img)
cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/opencv-logo.png')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 127, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

# Contours is a python list of all the contours in the image
# Each individual contour is a numpy array of (x,y) coordinates of boundary points of the objects.

print("Number of contours = " + str(len(contours)))
#print(contours[0].shape)
#cv2.imshow('Image', img)
#cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### OpenCV에서 contours를 그릴 때

```
cv2.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]]) 
```


```
Parameters:	
    image – 원본 이미지
    contours – contours정보.
    contourIdx – contours list type에서 몇번째 contours line을 그릴 것인지. -1 이면 전체
    color – contours line color
    thickness – contours line의 두께. 음수이면 contours line의 내부를 채움.

Returns: image에 contours가 그려진 결과
```

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/shapes5.jpg')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(imgray, 0, 255, 0)
#contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

imgGrey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(imgGrey, 240, 255, cv2.THRESH_BINARY) #240
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

for contour in contours[1:]:
    approx = cv2.approxPolyDP(contour, 0.01* cv2.arcLength(contour, True), True)
    cv2.drawContours(img, [approx], 0, (0, 0, 255), 5)


# Contours is a python list of all the contours in the image
# Each individual contour is a numpy array of (x,y) coordinates of boundary points of the objects.

print("Number of contours = " + str(len(contours)))
print(contours[0].shape)
#print(contours)
cv2.imshow('Image', img)
#cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/shapes6.jpg')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(imgray, 240, 255, 0)
_, thresh = cv2.threshold(imgray, 240, 255, cv2.THRESH_BINARY) 
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print("Number of contours = " + str(len(contours)))
#print(contours[0])

cv2.drawContours(img, contours, -1, (0, 255, 0), 3) # -1 is contour index indicating all contours


cv2.imshow('Image', img)
#cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/opencv-logo.png')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(imgray, 127, 255, 0)
_, thresh = cv2.threshold(imgray, 200, 255, cv2.THRESH_BINARY) 
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print("Number of contours = " + str(len(contours)))
#print(contours[0])

cv2.drawContours(img, contours, -1, (0, 255, 255), 3) # -1 is contour index indicating all contours


cv2.imshow('Image', img)
#cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

contour index 0을 그려보자.

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/opencv-logo.png')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(imgray, 200, 255, 0)
_, thresh = cv2.threshold(imgray, 200, 255, cv2.THRESH_BINARY) 

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print("Number of contours = " + str(len(contours)))
print(contours[0])

cv2.drawContours(img, contours, 0, (255, 0, 255), 3) # -1 is contour index indicating all contours


cv2.imshow('Image', img)
#cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/opencv-logo.png')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(imgray, 200, 255, 0)
_, thresh = cv2.threshold(imgray, 200, 255, cv2.THRESH_BINARY) 

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print("Number of contours = " + str(len(contours)))
print(contours[0])

cv2.drawContours(img, contours, -1, (255, 255, 0), 3)
cv2.drawContours(imgray, contours, -1, (0, 0, 0), 3)

cv2.imshow('Image', img)
cv2.imshow('Image GRAY', imgray)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 24. Motion Detection and Tracking

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture('pic/vtest.avi')

while cap.isOpened():
    
    ret, frame = cap.read()

    cv2.imshow("inter", frame)

    if cv2.waitKey(40) == 27:
        break

cv2.destroyAllWindows()
cap.release()

움직이는 object에 사각형을 그려보자.

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture('pic/vtest.avi')
frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))

frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc('X','V','I','D')

out = cv2.VideoWriter("output.avi", fourcc, 5.0, (1280,720))

ret, frame1 = cap.read()
ret, frame2 = cap.read()
print(frame1.shape)

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    cv2.drawContours(frame1, contours, -1, (0,255,0),2) # 2: thicjness
    
    cv2.imshow("feed", frame1)
    frame1 = frame2
    ret, frame2 = cap.read()

    if cv2.waitKey(40) == 27:
        break

cv2.destroyAllWindows()
cap.release()
out.release()

Noise를 제거하고, 사각형을 그리자.

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture('pic/vtest.avi')
frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))

frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc('X','V','I','D')

out = cv2.VideoWriter("output.avi", fourcc, 5.0, (1280,720))

ret, frame1 = cap.read()
ret, frame2 = cap.read()
print(frame1.shape)

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)

        if cv2.contourArea(contour) < 700:
            continue
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 3)
        
    #cv2.drawContours(frame1, contours, -1, (0, 255, 0), 2)

    cv2.imshow("feed", frame1)
    frame1 = frame2
    ret, frame2 = cap.read()

    if cv2.waitKey(40) == 27:
        break

cv2.destroyAllWindows()
cap.release()
out.release()

contourArea(contour) < 900 으로 바꾸자.

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture('pic/vtest.avi')
frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))

frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc('X','V','I','D')

out = cv2.VideoWriter("output.avi", fourcc, 5.0, (1280,720))

ret, frame1 = cap.read()
ret, frame2 = cap.read()
print(frame1.shape)

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)

        if cv2.contourArea(contour) < 900:
            continue
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 3)
        
    #cv2.drawContours(frame1, contours, -1, (0, 255, 0), 2)

    image = cv2.resize(frame1, (1280,720))
    out.write(image)
    cv2.imshow("feed", frame1)
    frame1 = frame2
    ret, frame2 = cap.read()

    if cv2.waitKey(40) == 27:
        break

cv2.destroyAllWindows()
cap.release()
out.release()

# 물체 따라가기

# 과제: 두개 이상의 물체 따라가기 
## 힌트: 9장 참고


## 25. Detect Simple Geometric Shapes 

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/shapes6.jpg')

cv2.imshow("shapes", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

img = cv2.imread('pic/shapes6.jpg')
imgGrey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thrash = cv2.threshold(imgGrey, 240, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(thrash, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

cv2.imshow("img", img)
for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.01* cv2.arcLength(contour, True), True)
    cv2.drawContours(img, [approx], 0, (0, 0, 0), 5)
    x = approx.ravel()[0]
    y = approx.ravel()[1] - 5
    if len(approx) == 3:
        cv2.putText(img, "Triangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
    elif len(approx) == 4:
        x1 ,y1, w, h = cv2.boundingRect(approx)
        aspectRatio = float(w)/h
        print(aspectRatio)
        if aspectRatio >= 0.95 and aspectRatio <= 1.05:
          cv2.putText(img, "square", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
        else:
          cv2.putText(img, "rectangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
    elif len(approx) == 5:
        cv2.putText(img, "Pentagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
    elif len(approx) == 10:
        cv2.putText(img, "Star", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
    else:
        cv2.putText(img, "Circle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))


cv2.imshow("shapes", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2.6 Understanding image Histograms

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = np.zeros((200,200), np.uint8)

cv2.imshow("img", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = np.zeros((200,200), np.uint8)

cv2.imshow("img", img)

plt.hist(img.ravel(), 256, [0, 256])
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = np.zeros((200,200), np.uint8)
cv2.rectangle(img, (0, 100), (200, 200), (255), -1)

cv2.imshow("img", img)


plt.hist(img.ravel(), 256, [0, 256])
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = np.zeros((200,200), np.uint8)
cv2.rectangle(img, (0, 100), (200, 200), (255), -1)
cv2.rectangle(img, (0, 50), (100, 100), (127), -1)

cv2.imshow("img", img)


plt.hist(img.ravel(), 256, [0, 256])
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = cv2.imread("pic/lena.jpg",0)

cv2.imshow("img", img)


plt.hist(img.ravel(), 256, [0, 256])
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = cv2.imread("pic/lena.jpg")

b, g, r = cv2.split(img)

cv2.imshow("img", img)
cv2.imshow("b", b)
cv2.imshow("g", g)
cv2.imshow("r", r)

plt.hist(b.ravel(), 256, [0, 256])
plt.hist(g.ravel(), 256, [0, 256])
plt.hist(r.ravel(), 256, [0, 256])

plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = cv2.imread("pic/lena.jpg",0)

hist = cv2.calcHist([img], [0], None, [256], [0, 256])
plt.plot(hist)
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = cv2.imread("pic/lena.jpg")

hist = cv2.calcHist([img], [0], None, [256], [0, 256]) #blue
plt.plot(hist)
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

## 27. Template matching 

pattern matching

In [ ]:
import cv2
import numpy as np
img = cv2.imread("pic/messi5.jpg")
template = cv2.imread("pic/messi_face.jpg")

cv2.imshow("img", img)
cv2.imshow("template", template)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

img = cv2.imread("pic/messi5.jpg")
grey_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
template = cv2.imread("pic/messi_face.jpg", 0)

res = cv2.matchTemplate(grey_img, template, cv2.TM_CCORR_NORMED )
print(res)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

img = cv2.imread("pic/messi5.jpg")
grey_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
template = cv2.imread("pic/messi_face.jpg", 0)

w, h = template.shape[::-1]

res = cv2.matchTemplate(grey_img, template, cv2.TM_CCORR_NORMED )
print("res",res)
print("img",img.shape)
print("template",template.shape)
print("res",res.shape)

threshold = 0.8;
loc = np.where(res >= threshold)
print(loc)


cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

img = cv2.imread("pic/messi5.jpg")
grey_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
template = cv2.imread("pic/messi_face.jpg", 0)

w, h = template.shape[::-1]

res = cv2.matchTemplate(grey_img, template, cv2.TM_CCORR_NORMED )
print(res)

threshold = 0.97;
loc = np.where(res >= threshold)
print(loc)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 28. Hough Line Transform Theory

The Hough transform is a feature extraction technique used in image analysis, computer vision, and digital image processing.  
The purpose of the technique is to find imperfect instances of objects within a certain class of shapes by a voting procedure.   
This voting procedure is carried out in a parameter space, from which object candidates are obtained as local maxima in a so-called accumulator space that is explicitly constructed by the algorithm for computing the Hough transform.

## 이론: 추가

#### Hough Transform Algorithm

1. Edge detection, e.g. using the Canny edge detector.
2. Mapping of edge points to the Hough space and storage in an accumulator.
3. Interpretation of the accumulator to yield lines of infinite length. The interpretation is done by thresholding and possibly other constraints.
4. Conversion of infinite lines to finite lines

OpenCV implements two kinds of Hough Line Transforms:
* Standard Hough Transform (HoughLines method)
* Probabilistic Houfh Transform (HoughLinesP method)

## 29. Hough Line Transform using HoughLines method 

In [ ]:
import cv2
import numpy as np

img = cv2.imread('pic/sudoku.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow('image', img)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

img = cv2.imread('pic/sudoku.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)
cv2.imshow('edges', edges)
lines = cv2.HoughLines(edges, 1, np.pi / 180, 200) # result is a list of polar coordinates.

for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho # x coordinate of top left corner
    y0 = b * rho # y coordinate of top left corner
    # x1 stores the rounded off value of (r * cos(theta) - 1000 * sin(theta))
    x1 = int(x0 + 1000 * (-b))
    # y1 stores the rounded off value of (r * sin(theta)+ 1000 * cos(theta))
    y1 = int(y0 + 1000 * (a))
    # x2 stores the rounded off value of (r * cos(theta)+ 1000 * sin(theta))
    x2 = int(x0 - 1000 * (-b))
    # y2 stores the rounded off value of (r * sin(theta)- 1000 * cos(theta))
    y2 = int(y0 - 1000 * (a))
    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)


cv2.imshow('image', img)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

1000 을 하는 이유:  https://stackoverflow.com/questions/18782873/houghlines-transform-in-opencv

즉 1000 단위 만큼 양쪽으로 떨어진 두 점을 구하기 위함. 따라서 다른 숫자도 가능 하다.

cv2.HoughLines 의 문제점: 선분이 아니라 직선으로 나타난다.

## 30. Probabilistic Hough Transform using HoughLinesP

In [ ]:
import cv2
import numpy as np
img = cv2.imread('pic/sudoku.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
cv2.imshow('edges', edges)

lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=100,maxLineGap=10)

for line in lines:
    x1,y1,x2,y2 = line[0]
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow('image', img)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
img = cv2.imread('pic/road.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
cv2.imshow('edges', edges)

lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=100,maxLineGap=10)

for line in lines:
    x1,y1,x2,y2 = line[0]
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow('image', img)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

## 31. Road Lane Line Detection (Part 1)

In [ ]:
import matplotlib.pylab as plt
import cv2
import numpy as np
%matplotlib qt

image = cv2.imread('pic/road.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)
plt.show()

In [ ]:
import matplotlib.pylab as plt
import cv2
import numpy as np
%matplotlib qt

image = cv2.imread('pic/road.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

print(image.shape)
height = image.shape[0]
width = image.shape[1]

plt.imshow(image)
plt.show()

In [ ]:
import matplotlib.pylab as plt
import cv2
import numpy as np
%matplotlib qt

image = cv2.imread('pic/road.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

print(image.shape)
height = image.shape[0]
width = image.shape[1]

region_of_interest_vertices = [
    (0, height),
    (width/2, height/2),
    (width, height)
]

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    channel_count = img.shape[2]
    match_mask_color = (255,) * channel_count
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

cropped_image = region_of_interest(image,
                np.array([region_of_interest_vertices], np.int32),)

plt.imshow(cropped_image)
plt.show()

## 32. Road Lane Line Detection (Part 2)

In [ ]:
import matplotlib.pylab as plt
import cv2
import numpy as np
%matplotlib qt

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    channel_count = img.shape[2]
    match_mask_color = (255,) * channel_count
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

image = cv2.imread('pic/road.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

print(image.shape)
height = image.shape[0]
width = image.shape[1]

region_of_interest_vertices = [
    (0, height),
    (width/2, height/2),
    (width, height)
]

cropped_image = region_of_interest(image,
                np.array([region_of_interest_vertices], np.int32),)

gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_RGB2GRAY)
canny_image = cv2.Canny(gray_image, 100, 200)


plt.imshow(canny_image)
plt.show()

region of interest를 사용하니, 차선이 잘 인식 되었다.  
다음으로, region of interest 를 나타내는 삼각형의 외곽선을 제거하자.

In [ ]:
import matplotlib.pylab as plt
import cv2
import numpy as np
%matplotlib qt

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    #channel_count = img.shape[2]
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

image = cv2.imread('pic/road.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)
height = image.shape[0]
width = image.shape[1]

region_of_interest_vertices = [
    (0, height),
    (width/2, height/2),
    (width, height)
]

gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
canny_image = cv2.Canny(gray_image, 100, 200)
cropped_image = region_of_interest(canny_image,
                np.array([region_of_interest_vertices], np.int32),)

plt.imshow(cropped_image)
plt.show()

In [ ]:
import matplotlib.pylab as plt
import cv2
import numpy as np

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    #channel_count = img.shape[2]
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def drow_the_lines(img, lines):
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(blank_image, (x1,y1), (x2,y2), (0, 255, 0), thickness=2)

    img = cv2.addWeighted(img, 0.8, blank_image, 1, 0.0)
    return img

image = cv2.imread('pic/road.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)
height = image.shape[0]
width = image.shape[1]
region_of_interest_vertices = [
    (0, height),
    (width/2, height/2),
    (width, height)
]

gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
canny_image = cv2.Canny(gray_image, 100, 200)
cropped_image = region_of_interest(canny_image,
                np.array([region_of_interest_vertices], np.int32),)

lines = cv2.HoughLinesP(cropped_image,
                        rho=6,
                        theta=np.pi/180,
                        threshold=160,
                        lines=np.array([]),
                        minLineLength=40,
                        maxLineGap=25)

image_with_lines = drow_the_lines(image, lines)

plt.imshow(image_with_lines)
plt.show()

## 33. Road Lane Line Detection(Part 3)

for video frame

In [ ]:
import matplotlib.pylab as plt
import cv2
import numpy as np


def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    #channel_count = img.shape[2]
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def drow_the_lines(img, lines):
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(blank_image, (x1,y1), (x2,y2), (0, 255, 0), thickness=10)

    img = cv2.addWeighted(img, 0.8, blank_image, 1, 0.0)
    return img

# = cv2.imread('road.jpg')
#image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
def process(image):
    #print(image.shape)
    height = image.shape[0]
    width = image.shape[1]
    region_of_interest_vertices = [
        (0, height),
        (width/2, height/2),
        (width, height)
    ]
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    canny_image = cv2.Canny(gray_image, 100, 120)
    cropped_image = region_of_interest(canny_image,
                    np.array([region_of_interest_vertices], np.int32),)
    lines = cv2.HoughLinesP(cropped_image,
                            rho=2,
                            theta=np.pi/180,
                            threshold=50,
                            lines=np.array([]),
                            minLineLength=40,
                            maxLineGap=100)
    image_with_lines = drow_the_lines(image, lines)
    return image_with_lines

cap = cv2.VideoCapture('pic/test.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    frame = process(frame)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()